# Workshop 2: Introduction to `Snakemake` workflows, update on new Open-TYNDP features & benchmarking framework

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pypsa atlite pandas geopandas xarray matplotlib hvplot geoviews plotly highspy holoviews folium mapclassify snakemake
```
:::

In [ ]:
# uncomment for running this notebook on Colab
# !pip install pypsa atlite pandas geopandas xarray matplotlib hvplot geoviews plotly highspy holoviews folium mapclassify snakemake

In [ ]:
# import packages
from IPython.display import Code, SVG, Image, display
from urllib.request import urlretrieve
from matplotlib.ticker import MultipleLocator
import cartopy.crs as ccrs
import os
import zipfile
import numpy as np
import pypsa
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pypsa.plot.maps.static import (
    add_legend_circles,
    add_legend_patches,
    add_legend_lines,
)

In [ ]:
urls = {
    "data/data_raw.csv": "https://storage.googleapis.com/open-tyndp-data-store/workshop-02/data_raw.csv",
    "data/open-tyndp.zip": "https://storage.googleapis.com/open-tyndp-data-store/workshop-02/open-tyndp.zip",
    "data/network_NT_presolve_highres_2030.nc": "https://storage.googleapis.com/open-tyndp-data-store/workshop-02/network_NT_presolve_highres_2030.nc",
}

os.makedirs("data", exist_ok=True)
for name, url in urls.items():
    if os.path.exists(name):
        print(f"File {name} already exists. Skipping download.")
    else:
        print(f"Retrieving {name} from GCP storage.")
        urlretrieve(url, name)
        print(f"File available in {name}.")

to_dir="data/open-tyndp"
if not os.path.exists(to_dir):
    with zipfile.ZipFile("data/open-tyndp.zip", "r") as zip_ref:
        zip_ref.extractall(to_dir)
print(f"Open-TYNDP available in '{to_dir}'.")

print("Done")

# The `Snakemake` tool

<img src="snakemake_logo.png" width="300px" />

The `Snakemake` workflow management system is a tool to create reproducible and scalable data analyses.
Workflows are described via a human readable, Python based language. They can be seamlessly scaled to server, cluster, grid and cloud environments, without the need to modify the workflow definition.

Snakemake follows the [GNU Make](https://www.gnu.org/software/make) paradigm: workflows are defined in terms of so-called `rules` that define how to create a set of output files from a set of input files. Dependencies between the rules are determined automatically, creating a DAG (directed acyclic graph) of jobs that can be automatically parallelized.

:::{note}
Documentation for this package is available at https://snakemake.readthedocs.io/. You can also check out a [slide deck Snakemake Tutorial](https://slides.com/johanneskoester/snakemake-tutorial) by Johannes Köster (2024).

Mölder, F., Jablonski, K.P., Letcher, B., Hall, M.B., Tomkins-Tinch, C.H., Sochat, V., Forster, J., Lee, S., Twardziok, S.O., Kanitz, A., Wilm, A., Holtgrewe, M., Rahmann, S., Nahnsen, S., Köster, J., 2021. Sustainable data analysis with Snakemake. F1000Res 10, 33.
:::


## A minimal Snakemake example

To check out how this looks in practice, we've prepared a minimal Snakemake example workflow that processes some data. The minimal workflow consists of the following rules:
- `retrieve_data`
- `build_data`
- `prepare_network`
- `solve_network`
- `plot_benchmark`
- `all`

<div style="text-align: center;">
<img src="minimal_workflow.png" width="400px" />
</div>

We will first need to load the raw data file used in this minimal example into our google drive:

### The `Snakefile` and `rules`

The rules need to be defined in a so-called `Snakefile` that sits in the same directory as your current working directory. For our minimal example the `Snakefile` looks like this:

In [ ]:
Code(filename='Snakefile', language="Python")

### Calling a workflow

You can then execute the workflow by asking for the target file `data/benchmark.pdf` or any intermediate file:
```
snakemake -call data/benchmark.pdf
```

Alternatively you can also execute the workflow by calling a rule that produces an intermediate file:
```
snakemake -call build_data
```
NOTE: It is important that you cannot call a rule that includes a wildcard without specifying what the wildcard should be filled with. Otherwise Snakemake will not know what to propagate back.

Or you can call the common rule `all` which can be used to execute the entire workflow. It takes the final workflow output as its input and thus requires all previous dependent rules to be run as well:
```
snakemake -call all
```

A very important instrument is the `-n` flag which executes a `dry-run`. It is recommended to always first execute a `dry-run` before the actual execution of the workflow. This simply prints out the directed acyclic graph (DAG) of the workflow to investigate without actually executing it.

Let's try this out and investigate the output:

In [ ]:
! snakemake -call all -n

### Visualizing the `DAG` of a worflow

You can also visualize the `DAG` of jobs using the `--dag` flag and the Graphviz `dot` command. This will not run the workflow but only create the visualization:
```
snakemake -call all --dag | dot -Tsvg > dag.svg
```

In [ ]:
! snakemake -call all --dag | sed -n "/digraph/,\$p" | dot -Tpng > dag_minimal.png

In [ ]:
display(Image('dag_minimal.png'))

Alternatively, you can also visualize a filegraph like the figure above which includes also some information about the inputs and outputs to each of the rules.

You can reproduce the figure from above with the following command:
```
snakemake -call all --filegraph | dot -Tsvg > filegraph.svg
```

In [ ]:
! snakemake -call all --filegraph | sed -n "/digraph/,\$p" | dot -Tsvg > filegraph_minimal.svg

In [ ]:
display(SVG('filegraph_minimal.svg'))

## Task 1: Executing a workflow with Snakemake

a) For our minimal example, execute a `dry-run` to produce the intermediate file `data/base_2030.nc`.

b) Execute the workflow and investigate what happens if you try to execute the workflow again.

c) Delete the final output files `data/benchmark.pdf` and investigate what happens if you try to execute the workflow again.

d) Import the raw input data file `data/data_raw.csv` using pandas and save it again overwriting the original file. Investigate what happens if you try to execute the workflow again. <br>
Hint: Alternative you can also just `touch` the file by executing `from pathlib import Path` and `Path("data/data_raw.csv").touch()`

e ) Finally, open the `Snakefile` and add a second rule that processes the file `data_raw_2.csv` using the same script as the `build_data` rule. Add the output of this new rule as a second input to the `prepare_network` rule.

## Using Snakemake to launch the open-TYNDP workflow

We have already retrieved a prebuilt version of the `open-tyndp` GitHub repository into our working directory...

The `open-tyndp` contains the following structure (directories which might be of particular interest to you are marked in bold):

- **benchmarks**: will store snakemake benchmarks (does not exist initially)
- **config**: configurations used in the study
- cutouts: will store raw weather data cutouts from atlite (does not exist initially)
- **data**: includes input data that is not produced by any snakemake rule. Various different input files are retrieved from external storage and stored in this directory
- doc: includes all files necessary to build the readthedocs documentation of PyPSA-Eur
- **envs**: includes all the mamba environment specifications to run the workflow
- logs: will store log files (does not exist initially)
- **notebooks**: includes all the notebooks used for ad-hoc analysis
- report: contains all files necessary to build the report; plots and result files are generated automatically
- **rules**: includes all the snakemake rules loaded in the Snakefile
- **resources**: will store intermediate results of the workflow which can be picked up again by subsequent rules (does not exist initially)
- **results**: will store the solved PyPSA network data, summary files and plots (does not exist initially)
- **scripts**: includes all the Python scripts executed by the snakemake rules to build the model

We now need to change our working directory to this new directory:

In [ ]:
os.chdir('data/open-tyndp')

Let's check that we are indeed in the new directory now:

In [ ]:
os.getcwd()

We can now use Snakemake to call some of the rules to produce outputs with the `open-tyndp` PyPSA model. 

We will use the prepared TYNDP configuration file (`config/config.tyndp.yaml`) and schedule a dry-run with `-n` as we only want to investigate the DAG of the workflow:

In [ ]:
! snakemake -call all --configfile config/config.tyndp.yaml -n

The corresponding rule graph to this workflow will look like this:

In [ ]:
! snakemake -call all -F --rulegraph | sed -n "/digraph/,\$p" | dot -Tpng > rulegraph_open_tyndp.png

In [ ]:
display(Image("rulegraph_open_tyndp.png"))

As you can see this workflow is much more complicated than our minimal example from the beginning.

However, the general idea remains the same. We retrieve data wich we consequently process, then we prepare the model network and we solve it before we postprocess the results (summary, plotting, benchmarks).

:::{note}
If you are executing this notebook on your local machine, you can also use the `conda` package manager to install the `open-tyndp` environment and run the workflow instead of dry-runs:
```
conda env create --file envs/<YourSystemOS>-pinned.yaml
```
:::

## Task 2: Adjusting the Open-TYNDP workflow with the configuration file

a) Make some changes in the configuration file and call another **dry-run** of the `open-tyndp` model again to see the changes to the workflow.

# Update on new features

In [ ]:
# For the purposes of this workshop we will primarily focus on the National Trends (NT) scenario

## Time Series

In [ ]:
# read network
# --- Electricity demand
# --- PECD
# look for renewables components
# Explain / remind difference between time varying and fixed attributes, how to access them
# Plot both time of parameters
# Compare to Report

First, we will import the solved network for the National Trends (NT) scenario for 2030 and 2040

In [ ]:
# hourly networks
n_2030_NT_presolve_highres = pypsa.Network("../network_NT_presolve_highres_2030.nc")

### Electricity demand

We can then explore the electricity demand that is attached to the network. Can you remember how to access `Loads` timeseries in PyPSA?

Correct! You can use the `loads_t` key and its `p_set` attribute:

In [ ]:
loads_2030 = n_2030_NT_presolve_highres.loads_t.p_set
display(loads_2030)

Let's plot these electricity demand time series:

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(14, 7))

# Plot each load time series
for load in loads_2030.columns:
    ax.plot(loads_2030.index, loads_2030[load], label=load, linewidth=1.5, alpha=0.8)

# Formatting
ax.set_xlabel("Time", fontsize=12, fontweight='bold')
ax.set_ylabel("Load [MW]", fontsize=12, fontweight='bold')
ax.set_title("Electricity Load Time Series National Trends 2030", fontsize=14, fontweight='bold', pad=20)

# Grid
ax.grid(True, alpha=0.3, linestyle='--')

# Legend
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', 
          frameon=True, shadow=True, fontsize=9, ncols=6)
plt.tight_layout()
plt.xticks(rotation=45, ha='right');

This is very confusing to look at. Let's filter that down a bit...

In [ ]:
# group country profiles together
country_mapping = n_2030_NT_presolve_highres.buses.query("carrier=='AC'").country
loads_2030_by_country = (
    n_2030_NT_presolve_highres
    .loads_t
    .p_set.T
    .rename(country_mapping, axis=0)
    .groupby("Load").sum().T
    .query("index  >= '2009-03-01' and index <= '2009-03-07'")
)

# Create the plot
fig, ax = plt.subplots(figsize=(14, 7))

# Plot each load time series
for load in loads_2030_by_country.columns:
    ax.plot(loads_2030_by_country.index, loads_2030_by_country[load], label=load, linewidth=1.5, alpha=0.8)

# Formatting
ax.set_xlabel("Time", fontsize=12, fontweight='bold')
ax.set_ylabel("Load [MW]", fontsize=12, fontweight='bold')
ax.set_title("Electricity Load Time Series National Trends 2030 - March 1-7", fontsize=14, fontweight='bold', pad=20)

# Grid
ax.grid(True, alpha=0.3, linestyle='--')

# Legend
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', 
          frameon=True, shadow=True, fontsize=9, ncols=6)
plt.tight_layout()
plt.xticks(rotation=45, ha='right');

#### Task: Can you remember how to use the **PyPSA Statistics** module that we introduced in the last workshop to interactively visualize these electricity demand inputs from the network:

First we need to load a lower resolution network that was solved, so we can use the statistics module to analyse it.

In [ ]:
n_2030_NT_solved_lowres = pypsa.Network("results/tyndp/NT/networks/base_s_all___2030.nc")
n_2040_NT_solved_lowres = pypsa.Network("results/tyndp/NT/networks/base_s_all___2040.nc")

In [ ]:
# let's fill missing colors first
n_2030_NT_solved_lowres.carriers.loc["none", "color"] = "#000000"
n_2030_NT_solved_lowres.carriers.loc["", "color"] = "#000000"
# and define a helper variable
s = n_2030_NT_solved_lowres.statistics

In [ ]:
s.energy_balance(bus_carrier="low voltage", comps="Load", aggregate_time=False, groupby=False).T

In [ ]:
fig, ax, facet_col  = s.withdrawal.plot.area(
    bus_carrier="low voltage",
    y="value",
    x="snapshot",
    color="carrier",
    stacked=True,
    facet_row="country",
    query="carrier == 'electricity' and country in ['DE', 'FR']",
)
fig.suptitle('Electricity demand time series', y=1.05)
ax[0,0].set_ylabel("Load [MW]")
ax[1,0].set_ylabel("Load [MW]")
ax[1,0].set_xlabel("Time");

We can also look at all the countries at the same time...

In [ ]:
fig, ax, facet_col  = s.withdrawal.plot.line(
    bus_carrier="low voltage",
    y="value",
    x="snapshot",
    color="country",
)
fig.suptitle('Electricity Load Time Series National Trends 2030', y=1.05)
ax.set_ylabel("Load [MW]")
ax.set_xlabel("Time");

In [ ]:
# Task: Take a few minutes to investigate the electricity demand series with the pypsa.statistics module

### PECD capacity factors

The Pan European Climate Database (PECD) provides capacity factor profiles for all the different renewable technologies used in the TYNDP. We processed these input data files into a Python and PyPSA friendly input format.

Let's start by looking at the processed capacity factor time series for solar PV Utility for 2030

In [ ]:
cf_pv_util = pd.read_csv("resources/tyndp/NT/pecd_data_LFSolarPVRooftop_2030.csv", index_col=0, parse_dates=True)
display(cf_pv_util.head(10))

For one week, these capacity factors look like this:

In [ ]:
loads_2030_by_country = (
    cf_pv_util
    .query("index  >= '2009-03-01' and index <= '2009-03-07'")
)

# Set seaborn style
sns.set_style("whitegrid")

# Create the plot
fig, ax = plt.subplots(figsize=(14, 7))

# Plot each load time series
for load in loads_2030_by_country.columns:
    ax.plot(loads_2030_by_country.index, loads_2030_by_country[load], label=load, linewidth=1.5, alpha=0.8)

# Formatting
ax.set_xlabel("Time", fontsize=12, fontweight='bold')
ax.set_ylabel("Capacity Factor", fontsize=12, fontweight='bold')
ax.set_title("Capacity Factor Time Series National Trends 2030 - March 1-7", fontsize=14, fontweight='bold', pad=20)

# Grid
ax.grid(True, alpha=0.3, linestyle='--')

# Legend
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', 
          frameon=True, shadow=True, fontsize=9, ncols=6)
plt.tight_layout()
plt.xticks(rotation=45, ha='right');

In [ ]:
# Task: In the open-tyndp, offshore wind, onshore wind and solar RES are added to the model at this time. 
# Take some time to investigate the network and find the corresponding capacity factor time series.
# Hint: Capacity factors are implemented in PyPSA by using the per unit dispatch limit (`p_min_pu`)

## Onshore wind and solar

The TYNDP provides expansion trajectories for given investment candidates and expandable technologies. For the implemented Onshore wind and solar technologies, this has been included within this beta release v0.3 of the Open-TYNDP model.

It is possible to retrieve those values from the network. However, for simplicity reasons, we will import the values from the processed input files for the `DE` scenario directly to investigate the entire trajectory paths at once.

In [ ]:
trajectories = pd.read_csv("resources/tyndp/DE/tyndp_trajectories.csv")
trajectories

Similar to the capacity factor time series, we want to focus on the Solar PV Utility technology and their trajectory path. Let's take Germany (DE00) to investigate

In [ ]:
trajectories_pv_utility_de = (
    trajectories
    .query("carrier == 'solar-pv-utility' and bus == 'DE00'")
)

In [ ]:
def plot_trajectories(trajectory_df, title="Trajectories"):
    """
    Plot trajectories provided in DataFrame.
    """

    # Set seaborn style
    sns.set_style("whitegrid")

    # Create figure
    fig, ax = plt.subplots(figsize=(12, 7))

    # Single scenario
    trajectory_df = trajectory_df.sort_values('pyear')

    # Plot upper bound
    ax.plot(trajectory_df['pyear'], trajectory_df['p_nom_max'].div(1e3),
            label='Maximum Capacity (p_nom_max)', linewidth=2.5,
            linestyle='--', color='#E63946', marker='o')

    # Plot lower bound
    ax.plot(trajectory_df['pyear'], trajectory_df['p_nom_min'].div(1e3),
            label='Minimum Capacity (p_nom_min)', linewidth=2.5,
            linestyle='--', color='#1D3557', marker='o')

    # Fill between bounds
    ax.fill_between(trajectory_df['pyear'],
                    trajectory_df['p_nom_min'].div(1e3),
                    trajectory_df['p_nom_max'].div(1e3),
                    alpha=0.25, color='#457B9D', label='Trajectory Range')

    # Formatting
    ax.set_xlabel("Planning Year", fontsize=13, fontweight='bold')
    ax.set_ylabel("Nominal Capacity [GW]", fontsize=13, fontweight='bold')
    ax.set_title(title,
                 fontsize=15, fontweight='bold', pad=20)

    # Legend
    ax.legend(loc='best', frameon=True, shadow=True, fontsize=11)

    # Grid
    ax.grid(True, alpha=0.4, linestyle='-', linewidth=0.5)

    # Format x-axis to show years as integers
    ax.xaxis.set_major_locator(MultipleLocator(5))

    plt.tight_layout()
    plt.show()

In [ ]:
plot_trajectories(trajectories_pv_utility_de, title="Solar PV Utility Capacity Trajectories DE scenario - DE00")

aNow, let's access the network of the DE scenario to compare one of these values for 2040.

In [ ]:
# TODO: include DE scenarios in open-tyndp.zip
n_2040_DE_solved_lowres = pypsa.Network("results/tyndp/DE/networks/base_s_all___2040.nc")

In [ ]:
trajectories_pv_utility_de_from_network = (
    n_2040_DE_solved_lowres
    .generators
    .query("carrier == 'solar-pv-utility' and bus == 'DE00'")
    [["p_nom", "p_nom_min", "p_nom_max"]]
    .div(1e3)  # in GW
)
trajectories_pv_utility_de_from_network

As we can see, the `p_nom_max` and `p_nom_min` values for 2040 do not match directly with the reported trajectories value analysed above. This is because each new Generator will have set trajectories that correspond to the new cummulatively installed capacities taking into account optimization results from previous years. So if we add up the existing capacity (`p_nom`) from 2030 and `p_nom_max` and `p_nom_min` from 2040, we will find the reported trajectory values from before again:

In [ ]:
trajectories_pv_utility_de_from_network.loc["DE00 0 solar-pv-utility-2040", ["p_nom_min", "p_nom_max"]] + trajectories_pv_utility_de_from_network.loc["DE00 0 solar-pv-utility-2030", "p_nom"]

In [ ]:
# Task: Reproduce this exercise for Onshore Wind. You can copy and reuse the existing code used above.

## Offshore Hubs

In [ ]:
# read network
# plot OH hubs maps
# Constraitns
# capacities
# questions

## H2 imports

In [ ]:
# read network
# plot H2 map with import corridors
# table to showcase the corridors
# optional: Task to investigate values

There have also been some important additions to H2 infrastructure since our last workshop. The different H2 import corridors are now included in the model with a simple pipeline transport model representation such as for H2 reference grid.

We can investigate our National Trends network from before to find a similar plot we showed last time. Let's define some (quite extensive) plotting functions from the open-tyndp workflow:

In [ ]:
def group_import_corridors(df):
    """
    Group pipes which connect same buses and return overall capacity.
    """
    df = df.copy()

    # there are pipes for each investment period rename to AC buses name for plotting
    df["index_orig"] = df.index
    df.rename(index=lambda x: x.split(" - ")[0], inplace=True)
    return df.groupby(level=0).agg(
        {"p_nom": "sum", "p_nom_opt": "sum", "index_orig": "first"}
    )


def plot_h2_map_base(
    network, 
    map_opts, 
    proj,
    figsize=(12, 12), 
    expanded=False, 
    regions_for_storage=None,     
    color_h2_pipe = "#499a9c",
    color_h2_imports = "#FFA500",
    color_h2_node = "#ff29d9",
):
    """
    Plots the base hydrogen network pipelines capacities, hydrogen buses and import potentials.
    If expanded is enabled, the optimal capacities are plotted instead.
    If regions are given, hydrogen storage capacities are plotted for those regions with aggregated H2 tank storage
    and underground H2 cavern capacities.

    Parameters
    ----------
    network : pypsa.Network
        PyPSA network for plotting the hydrogen grid. Can be either presolving or post solving.
    map_opts : dict
        Map options for plotting.
    expanded : bool, optional
        Whether to plot expanded capacities. Defaults to plotting only base network (p_nom).
    regions_for_storage : gpd.GeoDataframe, optional
        Geodataframe of regions to use for plotting hydrogen storage capacities. Index needs to match storage locations.
        If none is given, no hydrogen storage capacities are plotted.

    Returns
    -------
    None
        Saves the map plot as figure.
    """
    n = network.copy()

    linewidth_factor = 4e3

    n.links.drop(
        n.links.index[
            ~(
                n.links.carrier.str.contains("H2 pipeline")
                | n.links.carrier.str.contains("H2 import")
            )
        ],
        inplace=True,
    )
    n.links.drop(
        n.links.index[n.links.carrier.str.contains("OH")],
        inplace=True,
    )

    p_nom = "p_nom_opt" if expanded else "p_nom"
    # capacity of pipes and imports
    h2_pipes = n.links[n.links.carrier == "H2 pipeline"][p_nom]
    h2_imports = n.links[n.links.carrier.str.contains("H2 import")]

    # group high and low import corridors together
    h2_imports = group_import_corridors(h2_imports)[p_nom]
    n.links.rename(index=lambda x: x.split(" - ")[0], inplace=True)
    # group links by summing up p_nom values and taking the first value of the rest of the columns
    other_cols = dict.fromkeys(n.links.columns.drop(["p_nom_opt", "p_nom"]), "first")
    n.links = n.links.groupby(level=0).agg(
        {"p_nom_opt": "sum", "p_nom": "sum", **other_cols}
    )

    # set link widths
    link_widths_pipes = h2_pipes / linewidth_factor
    link_widths_imports = h2_imports / linewidth_factor
    if link_widths_pipes.notnull().empty:
        print("No base H2 pipeline capacities to plot.")
        return
    link_widths_pipes = link_widths_pipes.reindex(n.links.index).fillna(0.0)
    link_widths_imports = link_widths_imports.reindex(n.links.index).fillna(0.0)

    # drop non H2 buses
    n.buses.drop(
        n.buses.index[
            (~n.buses.carrier.str.contains("H2")) | (n.buses.carrier.str.contains("OH"))
        ],
        inplace=True,
    )

    # optionally add hydrogen storage capacities onto the map
    if regions_for_storage is not None:
        h2_storage = n.stores.query("carrier.str.contains('H2')")
        regions_for_storage["H2"] = (
            h2_storage.rename(index=h2_storage.bus.map(n.buses.location))
            .e_nom_opt.groupby(level=0)
            .sum()
            .div(1e6)
        )  # TWh
        regions_for_storage["H2"] = regions_for_storage["H2"].where(
            regions_for_storage["H2"] > 0.1
        )

    # plot H2 pipeline capacities and imports
    print("Plotting base H2 pipeline and import capacities.")
    fig, ax = plt.subplots(figsize=figsize, subplot_kw={"projection": proj})

    n.plot.map(
        geomap=True,
        bus_sizes=0.1,
        bus_colors=color_h2_node,
        link_colors=color_h2_pipe,
        link_widths=link_widths_pipes,
        branch_components=["Link"],
        ax=ax,
        **map_opts,
    )

    if regions_for_storage is not None:
        regions_for_storage = regions_for_storage.to_crs(proj.proj4_init)
        regions_for_storage.plot(
            ax=ax,
            column="H2",
            cmap="Blues",
            linewidths=0,
            legend=True,
            vmax=6,
            vmin=0,
            legend_kwds={
                "label": "Hydrogen Storage [TWh]",
                "shrink": 0.7,
                "extend": "max",
            },
        )

    if not h2_imports.empty:
        n.plot.map(
            geomap=True,
            bus_sizes=0,
            link_colors=color_h2_imports,
            link_widths=link_widths_imports,
            branch_components=["Link"],
            ax=ax,
            **map_opts,
        )

    sizes = [30, 10]
    labels = [f"{s} GW" for s in sizes]
    scale = 1e3 / linewidth_factor
    sizes = [s * scale for s in sizes]

    legend_kw = dict(
        loc="upper left",
        bbox_to_anchor=(0.32, 1.13),
        frameon=False,
        ncol=1,
        labelspacing=0.8,
        handletextpad=1,
    )

    add_legend_lines(
        ax,
        sizes,
        labels,
        patch_kw=dict(color="lightgrey"),
        legend_kw=legend_kw,
    )

    legend_kw = dict(
        loc="upper left",
        bbox_to_anchor=(0.55, 1.13),
        labelspacing=0.8,
        handletextpad=0,
        frameon=False,
    )

    add_legend_circles(
        ax,
        sizes=[0.2],
        labels=["H2 Node"],
        srid=n.srid,
        patch_kw=dict(facecolor=color_h2_node),
        legend_kw=legend_kw,
    )

    colors = (
        [color_h2_pipe, color_h2_imports] if not h2_imports.empty else [color_h2_pipe]
    )
    labels = ["H2 Pipeline", "H2 import"] if not h2_imports.empty else ["H2 Pipeline"]

    legend_kw = dict(
        loc="upper left",
        bbox_to_anchor=(0, 1.13),
        ncol=1,
        frameon=False,
    )

    add_legend_patches(ax, colors, labels, legend_kw=legend_kw)

    ax.set_facecolor("white")

    plt.show()

And plot the H2 reference grid together with the import corridors:

In [ ]:
n = n_2030_NT_solved_lowres.copy()

def load_projection(plotting_params):
    proj_kwargs = plotting_params.get("projection", dict(name="EqualEarth"))
    proj_func = getattr(ccrs, proj_kwargs.pop("name"))
    return proj_func(**proj_kwargs)


proj = load_projection(dict(name="EqualEarth"))

map_opts = {
    "boundaries": [-11, 30, 34, 71],
    "geomap_colors": {
        "ocean": "white",
        "land": "white",
    },
}

if n.buses.country.isin(["MA", "DZ"]).any():
    map_opts["boundaries"] = list(np.add(map_opts["boundaries"], [0, 0, -6, 0]))

plot_h2_map_base(n, map_opts, proj)

# Benchmarking framework

In [ ]:
# Present metrics used (incl. reference to methodology)
# Data sources used for comparison
# Mention introduction of onwind and solar
# Showcase current status
# --- Table
# --- Graphs

# Wrap up

In [ ]:
# Collect feeback via Slido